In [1]:
import random
import numpy as np

In [2]:
#
# Use e.g. https://www.compilejava.net/
#
#import java.util.*;
#import java.math.*;
#
#public class Entrypoint
#{
#  public static void main(String[] args)
#  {
#    BigInteger q = BigInteger.probablePrime(128, new Random());    
#    BigInteger inverse = new BigInteger("10000000000").modInverse(q);
#    System.out.println(q);
#    System.out.println(inverse);
#  }
#}

BASE = 10

#PRECISION_INTEGRAL = 1
#PRECISION_FRACTIONAL = 6
#Q = 10000019

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

In [3]:
def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

In [4]:
def share(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def reconstruct(sharing):
    return sum(sharing) % Q

def reshare(x):
    Y = [ share(x[0]), share(x[1]), share(x[2]) ]
    return [ sum(row) % Q for row in zip(*Y) ]

In [5]:
def add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]
    
def imul(x, k):
    return [ (xi * k) % Q for xi in x ]

In [6]:
INVERSE = 104491423396290281423421247963055991507 # inverse of BASE**FRACTIONAL_PRECISION
KAPPA = 6 # leave room for five digits overflow before leakage

assert((INVERSE * BASE**PRECISION_FRACTIONAL) % Q == 1)
assert(Q > BASE**(2*PRECISION + KAPPA))

def truncate(a):
    # map to the positive range
    b = add(a, [BASE**(2*PRECISION+1), 0, 0])
    # apply mask known only by P0, and reconstruct masked b to P1 or P2
    mask = random.randrange(Q) % BASE**(PRECISION + PRECISION_FRACTIONAL + KAPPA)
    mask_low = mask % BASE**PRECISION_FRACTIONAL
    b_masked = reconstruct(add(b, [mask, 0, 0]))
    # extract lower digits
    b_masked_low = b_masked % BASE**PRECISION_FRACTIONAL
    b_low = sub(share(b_masked_low), share(mask_low))
    # remove lower digits
    c = sub(a, b_low)
    # remove extra scaling factor
    d = imul(c, INVERSE)
    return d

In [7]:
def mul(x, y):
    # local computation
    z0 = (x[0]*y[0] + x[0]*y[1] + x[1]*y[0]) % Q
    z1 = (x[1]*y[1] + x[1]*y[2] + x[2]*y[1]) % Q
    z2 = (x[2]*y[2] + x[2]*y[0] + x[0]*y[2]) % Q
    # reshare and distribute
    Z = [ share(z0), share(z1), share(z2) ]
    w = [ sum(row) % Q for row in zip(*Z) ]
    # bring precision back down from double to single
    v = truncate(w)
    return v

In [8]:
class SecureRational(object):
    
    def secure(secret):
        z = SecureRational()
        z.shares = share(encode(secret))
        return z
    
    def reveal(self):
        return decode(reconstruct(reshare(self.shares)))
    
    def __repr__(self):
        return "SecureRational(%f)" % self.reveal()
    
    def __add__(x, y):
        z = SecureRational()
        z.shares = add(x.shares, y.shares)
        return z
    
    def __sub__(x, y):
        z = SecureRational()
        z.shares = sub(x.shares, y.shares)
        return z
    
    def __mul__(x, y):
        z = SecureRational()
        z.shares = mul(x.shares, y.shares)
        return z
    
    def __pow__(x, e):
        z = SecureRational.secure(1)
        for _ in range(e):
            z = z * x
        return z

In [9]:
x = SecureRational.secure(.5)
y = SecureRational.secure(-.25)
z = x * y
assert(z.reveal() == (.5) * (-.25))

In [10]:
class OpenRational(object):
    
    def __init__(self, secret):
        self.secret = secret
    
    def secure(secret):
        return OpenRational(secret)
    
    def reveal(self):
        return self.secret
    
    def __repr__(self):
        return "OpenRational(%f)" % self.reveal()
    
    def __add__(x, y):
        return OpenRational(x.secret + y.secret)
    
    def __sub__(x, y):
        return OpenRational(x.secret - y.secret)
    
    def __mul__(x, y):
        return OpenRational(x.secret * y.secret)
    
    def __pow__(x, e):
        z = OpenRational(1)
        for _ in range(e):
            z = z * x
        return z

In [11]:
# helper functions to map array of numbers to and from secure data type
secure = np.vectorize(lambda x: SecureRational.secure(x))
reveal = np.vectorize(lambda x: x.reveal())

In [12]:
class TwoLayerNetwork:

    def __init__(self, sigmoid):
        self.sigmoid = sigmoid
    
    def train(self, X, y, iterations=1000, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights 
        self.synapse0 = secure(2 * np.random.random((3,1)) - 1)

        # training
        for i in range(iterations):

            # forward propagation
            layer0 = X
            layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))

            # back propagation
            layer1_error = y - layer1
            layer1_delta = layer1_error * self.sigmoid.derive(layer1)
            
            # provide error feed by revealing values
            if (i+1) % (iterations//10) == 0:
                print("Error: %s" % np.mean(np.abs(reveal(layer1_error))))

            # update
            self.synapse0 += np.dot(layer0.T, layer1_delta) * alpha
            
    def predict(self, X):
        layer0 = X
        layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
        return layer1
    
    def print_weights(self):
        print("Layer 0 weights: \n%s" % self.synapse0)

In [13]:
class ThreeLayerNetwork:

    def __init__(self, sigmoid):
        self.sigmoid = sigmoid
    
    def train(self, X, y, iterations=1000, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights
        self.synapse0 = secure(2 * np.random.random((3,4)) - 1)
        self.synapse1 = secure(2 * np.random.random((4,1)) - 1)

        # training
        for i in range(iterations):
    
            # forward propagation
            layer0 = X
            layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
            layer2 = self.sigmoid.evaluate(np.dot(layer1, self.synapse1))

            # back propagation
            layer2_error = y - layer2
            layer2_delta = layer2_error * self.sigmoid.derive(layer2)
            layer1_error = np.dot(layer2_delta, self.synapse1.T)
            layer1_delta = layer1_error * self.sigmoid.derive(layer1)

            # provide error feedback by revealing values
            if (i+1) % (iterations//10) == 0:
                print("Error: %s" % np.mean(np.abs(reveal(layer2_error))))

            # update
            self.synapse1 += np.dot(layer1.T, layer2_delta) * alpha
            self.synapse0 += np.dot(layer0.T, layer1_delta) * alpha
            
    def predict(self, X):
        layer0 = X
        layer1 = self.sigmoid.evaluate(np.dot(layer0, self.synapse0))
        layer2 = self.sigmoid.evaluate(np.dot(layer1, self.synapse1))
        return layer2
    
    def print_weights(self):
        print("Layer 0 weights: \n%s" % self.synapse0)
        print("Layer 1 weights: \n%s" % self.synapse1)

In [14]:
np.array([2]) * np.array([1,2,3,4])

array([2, 4, 6, 8])

In [15]:
class SigmoidMaclaurin3:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(1/2)
        W1  = SecureRational.secure(1/4)
        W3  = SecureRational.secure(-1/48)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3))
        self.sigmoid_deriv = np.vectorize(lambda x: (ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [16]:
class SigmoidMaclaurin5:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(1/2)
        W1  = SecureRational.secure(1/4)
        W3  = SecureRational.secure(-1/48)
        W5  = SecureRational.secure(1/480)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [17]:
class SigmoidMaclaurin9:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(1/2)
        W1  = SecureRational.secure(1/4)
        W3  = SecureRational.secure(-1/48)
        W5  = SecureRational.secure(1/480)
        W7  = SecureRational.secure(-17/80640)
        W9  = SecureRational.secure(31/1451520)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5) + (x**7 * W7) + (x**9 * W9))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [18]:
class SigmoidInterpolated10:
    
    def __init__(self):
        ONE = SecureRational.secure(1)
        W0  = SecureRational.secure(0.5)
        W1  = SecureRational.secure(0.2159198015)
        W3  = SecureRational.secure(-0.0082176259)
        W5  = SecureRational.secure(0.0001825597)
        W7  = SecureRational.secure(-0.0000018848)
        W9  = SecureRational.secure(0.0000000072)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5) + (x**7 * W7) + (x**9 * W9))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [19]:
X_full = np.array([
            [0,0,0],
            [0,0,1],
            [0,1,0],
            [0,1,1],
            [1,0,0],
            [1,0,1],
            [1,1,0],
            [1,1,1]
        ])

def evaluate(network):
    for x in X_full:
        score = reveal(network.predict(secure(x)))
        prediction = 1 if score > 0.5 else 0
        print("Prediction on %s: %s (%.8f)" % (x, prediction, score))

In [20]:
X_train = np.array([
            [0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]
        ])

y_train = np.array([[
            0,
            0,
            1,
            1
        ]]).T

In [21]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = SigmoidMaclaurin5()

# train
network = TwoLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 10000)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 0.00539115
Error: 0.0025606125
Error: 0.00167358
Error: 0.001241815
Error: 0.00098674
Error: 0.000818415
Error: 0.0006990725
Error: 0.0006100825
Error: 0.00054113
Error: 0.0004861775
Layer 0 weights: 
[[SecureRational(4.974135)]
 [SecureRational(-0.000854)]
 [SecureRational(-2.486387)]]
Prediction on [0 0 0]: 0 (0.50000000)
Prediction on [0 0 1]: 0 (0.00066431)
Prediction on [0 1 0]: 0 (0.49978657)
Prediction on [0 1 1]: 0 (0.00044076)
Prediction on [1 0 0]: 1 (5.52331855)
Prediction on [1 0 1]: 1 (0.99969213)
Prediction on [1 1 0]: 1 (5.51898314)
Prediction on [1 1 1]: 1 (0.99946841)


In [22]:
X_train = np.array([
            [0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]
        ])

y_train = np.array([[
            0,
            1,
            1,
            0
        ]]).T

In [23]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = SigmoidMaclaurin5()

# train
network = TwoLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 1000)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 0.500000005
Error: 0.5
Error: 0.5000000025
Error: 0.5000000025
Error: 0.5
Error: 0.5
Error: 0.5
Error: 0.5
Error: 0.5
Error: 0.5
Layer 0 weights: 
[[SecureRational(0.000000)]
 [SecureRational(0.000000)]
 [SecureRational(0.000000)]]
Prediction on [0 0 0]: 0 (0.50000000)
Prediction on [0 0 1]: 0 (0.50000000)
Prediction on [0 1 0]: 0 (0.50000000)
Prediction on [0 1 1]: 0 (0.50000000)
Prediction on [1 0 0]: 0 (0.50000000)
Prediction on [1 0 1]: 0 (0.50000000)
Prediction on [1 1 0]: 0 (0.50000000)
Prediction on [1 1 1]: 0 (0.50000000)


In [24]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = SigmoidMaclaurin5()

# train
network = ThreeLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 100)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 0.496326875
Error: 0.4963253375
Error: 0.50109445
Error: 4.50917445533e+22
Error: 4.20017387687e+22
Error: 4.38235385094e+22
Error: 4.65389939428e+22
Error: 4.25720845129e+22
Error: 4.50520005372e+22
Error: 4.31568874384e+22
Layer 0 weights: 
[[SecureRational(970463188850515564822528.000000)
  SecureRational(1032362386093871682551808.000000)
  SecureRational(1009706886834648285970432.000000)
  SecureRational(852352894255113084862464.000000)]
 [SecureRational(999182403614802557534208.000000)
  SecureRational(747418473813466924711936.000000)
  SecureRational(984098986255565992230912.000000)
  SecureRational(865284701475152213311488.000000)]
 [SecureRational(848400149667429499273216.000000)
  SecureRational(871252067688430631387136.000000)
  SecureRational(788722871059090631557120.000000)
  SecureRational(868480811373827731750912.000000)]]
Layer 1 weights: 
[[SecureRational(818092877308528183738368.000000)]
 [SecureRational(940782003999550335877120.000000)]
 [SecureRational(9098825

In [25]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = SigmoidMaclaurin9()

# train
network = ThreeLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 100)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 0.49546145
Error: 0.4943132225
Error: 0.49390536
Error: 0.50914575
Error: 7.29251498137e+22
Error: 7.97702462371e+22
Error: 7.01752029207e+22
Error: 7.41001528681e+22
Error: 7.33032620012e+22
Error: 7.3022511184e+22
Layer 0 weights: 
[[SecureRational(867589421903116943491072.000000)
  SecureRational(925728055684076836749312.000000)
  SecureRational(822088442598816439336960.000000)
  SecureRational(755780148365930251943936.000000)]
 [SecureRational(884209157989405999235072.000000)
  SecureRational(792575791450471840874496.000000)
  SecureRational(923048733641244892200960.000000)
  SecureRational(806331401937938901106688.000000)]
 [SecureRational(798596437496442344439808.000000)
  SecureRational(823212471429560140824576.000000)
  SecureRational(864724012603872064307200.000000)
  SecureRational(808596872646821102485504.000000)]]
Layer 1 weights: 
[[SecureRational(769124640640082141773824.000000)]
 [SecureRational(836496252705041167679488.000000)]
 [SecureRational(807253860823596435

In [26]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = SigmoidMaclaurin3()

# train
network = ThreeLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 500)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 0.4821573275
Error: 0.46344183
Error: 0.4428059575
Error: 0.4168092675
Error: 0.388153325
Error: 0.3619875475
Error: 0.3025045425
Error: 0.2366579675
Error: 0.19651228
Error: 0.1748352775
Layer 0 weights: 
[[SecureRational(1.455894) SecureRational(1.376838)
  SecureRational(-1.445690) SecureRational(-2.383619)]
 [SecureRational(-0.794408) SecureRational(-2.069235)
  SecureRational(-1.870023) SecureRational(-1.734243)]
 [SecureRational(0.712099) SecureRational(-0.688947)
  SecureRational(0.740605) SecureRational(2.890812)]]
Layer 1 weights: 
[[SecureRational(-2.893681)]
 [SecureRational(6.238205)]
 [SecureRational(-7.945379)]
 [SecureRational(4.674321)]]
Prediction on [0 0 0]: 1 (0.50918230)
Prediction on [0 0 1]: 0 (0.16883382)
Prediction on [0 1 0]: 0 (0.40589161)
Prediction on [0 1 1]: 1 (0.82447640)
Prediction on [1 0 0]: 1 (0.83164009)
Prediction on [1 0 1]: 1 (0.83317334)
Prediction on [1 1 0]: 1 (0.74354671)
Prediction on [1 1 1]: 0 (0.18736629)


In [27]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = SigmoidMaclaurin3()

# train
network = ThreeLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 550)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 0.4804166425
Error: 0.45947967
Error: 0.4357743725
Error: 0.4050500125
Error: 0.3753641875
Error: 0.3339848675
Error: 0.253083505
Error: 0.202983755
Error: 0.1760496125
Error: 4.68435851895e+22
Layer 0 weights: 
[[SecureRational(400992477069057112145920.000000)
  SecureRational(405257358037106569510912.000000)
  SecureRational(442905398473782651256832.000000)
  SecureRational(377854517438411771478016.000000)]
 [SecureRational(431147901530714740359168.000000)
  SecureRational(478469347392975582265344.000000)
  SecureRational(384074406461635202908160.000000)
  SecureRational(418307054827778709913600.000000)]
 [SecureRational(388276373567757103398912.000000)
  SecureRational(427441225610272262062080.000000)
  SecureRational(370061894706638499086336.000000)
  SecureRational(451299387791588817108992.000000)]]
Layer 1 weights: 
[[SecureRational(412762217581329906663424.000000)]
 [SecureRational(342080083074562152988672.000000)]
 [SecureRational(379803268474628961468416.000000)]
 [Secu

In [28]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# pick approximation
sigmoid = SigmoidInterpolated10()

# train
network = ThreeLayerNetwork(sigmoid)
network.train(secure(X_train), secure(y_train), 10000)
network.print_weights()

# evaluate predictions
evaluate(network)

Error: 0.0384136825
Error: 0.01946007
Error: 0.0141456075
Error: 0.0115575225
Error: 0.010008035
Error: 0.0089747225
Error: 0.0082400825
Error: 0.00769687
Error: 0.007286195
Error: 0.00697363
Layer 0 weights: 
[[SecureRational(3.208028) SecureRational(3.359444)
  SecureRational(-3.632461) SecureRational(-4.094379)]
 [SecureRational(-1.552827) SecureRational(-4.403901)
  SecureRational(-3.997194) SecureRational(-3.271171)]
 [SecureRational(0.695226) SecureRational(-1.560569)
  SecureRational(1.758733) SecureRational(5.425429)]]
Layer 1 weights: 
[[SecureRational(-4.674311)]
 [SecureRational(5.910466)]
 [SecureRational(-9.854162)]
 [SecureRational(6.508941)]]
Prediction on [0 0 0]: 0 (0.28170669)
Prediction on [0 0 1]: 0 (0.00638341)
Prediction on [0 1 0]: 0 (0.33542098)
Prediction on [0 1 1]: 1 (0.99287968)
Prediction on [1 0 0]: 1 (0.74297185)
Prediction on [1 0 1]: 1 (0.99361066)
Prediction on [1 1 0]: 0 (0.03599433)
Prediction on [1 1 1]: 0 (0.00800036)
